<a href="https://colab.research.google.com/github/vilsonrodrigues/Projeto_Orientacao_Academica/blob/master/Projeto_Orientacao_Academica_Tratamento_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Por @vilsonrodrigues



# <center><font color='blue'>Projeto de Orientação Acadêmica</font>




<img src='https://ufrn.br/resources/documentos/identidadevisual/logotipo/logotipo_flat.png'>

## Parte I - Apresentação do Problema
### [1. Apresentação do Problema](#apresentacao)
## Parte II - Preparativos Iniciais
### [1. Bibliotecas](#bibliotecas)
### [2. Classe](#classe)
### [3. Funções](#funcoes)
### [4. Leituras da Bases](#leiturabase)
## Parte III - Desenvolvimento
### [1. Entendimento de Negócio](#negocio)
### [2. Entendimento de Dados](#enten_dados)
### [3. Preparação de Dados](#preparo)
### [4. Métricas de Avaliação](#metricas)
### [5. Modelagem](#modelagem)
### [6. Avaliação de Modelo](#avaliacao)
### [7. Conclusão e Recomendações](#conclusao)
### [8. Implantação de Modelos](#implantacao)


# Parte I - Apresentação do Problema

### **Meta**
Nossa meta é prever através de Machine Learning as probabilidades de um estudante ter sucesso em uma disciplina dado o histórico escolar na Universidade.

Nossos dados foram transformados para ficarem de uma forma génerica, assim, não dependemos de saber qual foi a matéria cursada anteriormente.

Com essa plataforma, os professores que orientam alunos, terão mais segurança e uma comprovação com base no Machine Learning que estudantes que tiveram um histórico semelhante e seguiram esse caminho tiveram mais êxito do que seguir por este outro.

Os dados foram coletados do período de 2015.1 até 2019.1.

**Esse notebook foca totalmente em analisar os dados e montar uma estratégia  de gerar dados genéricos, de modo que fiquem independente de qual disciplina o aluno curso no semeste anterior.**

O trabalho continua neste [notebook](https://), onde são gerados modelos de Machine Learning com dados genéricos.

### **Fonte dos Dados**
Dados foram extraídos do portal SIGAA e fornecidos pela Escola de Ciências e Tecnlogia (ECT).

### **Referência**
Hands On: Machine Learning with Scikit-Learning e Tensor Flow



# Parte II - Preparativos Iniciais

## 1. Bibliotecas
<a id='bibliotecas'></a>
#### [1.1 Instalações](#instalacao)
#### [1.2 Importações](#importacao)


### 1.1. Instalação
<a id='instalacao'></a>

In [0]:
!pip install pandas -U
!pip install scikit-learn -U
!pip install numpy -U
!pip install joblib==0.14.1

### 1.2. Importação
<a id='importacao'></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

In [0]:
import pandas as pd
import numpy as np

#Persistência em disco
import joblib
import pickle

#Visualização
import seaborn as sns
import matplotlib.pyplot as plt

#Transformadores e Modelador
from sklearn.pipeline          import Pipeline
from sklearn.preprocessing     import StandardScaler
from sklearn.base              import BaseEstimator, TransformerMixin

#Selecao de modelo
from sklearn.model_selection   import train_test_split
from sklearn.model_selection   import KFold
from sklearn.model_selection   import cross_val_score
from sklearn.model_selection   import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import chi2

#Metricas
from sklearn.metrics           import classification_report
from sklearn.metrics           import confusion_matrix
from sklearn.metrics           import make_scorer
from sklearn.metrics           import accuracy_score
from sklearn.metrics           import f1_score
from sklearn.metrics           import recall_score
from sklearn.metrics           import precision_score

#Classificadores
from sklearn.ensemble          import RandomForestClassifier
from xgboost                   import XGBClassifier
from sklearn.tree              import DecisionTreeClassifier
from sklearn.neighbors         import KNeighborsClassifier
from sklearn.naive_bayes       import GaussianNB
from sklearn.svm               import SVC
from sklearn.neural_network    import MLPClassifier

#modo não-perturbe
import warnings
warnings.filterwarnings("ignore")

##2. Classe
<a id='classe'></a>


Classe para realizar a geração de dados genéricos

In [0]:
class geraDadosGenericos(TransformerMixin):  
  def __init__(self, carga):
    self.carga = carga

	#nao irei fazer nenhum treinamento	


  def fit(self, X, y = None):
    return self    

  #-------------------------funcoes de geracao de dados da classe---------------------------------

  def filtraSituacoes(self, df_notas):
    #conversao para string
    df_notas['matricula'] = df_notas['matricula'].astype(str)
    df_notas['periodo']   = df_notas['periodo'].astype(str)

    #substitui todas as situacoes de reprovado e aprovado para ser apenas um tipo
    df_notas.loc[df_notas['situacao'].str.contains('REPROVADO'), 'situacao'] = 'REPROVADO'
    df_notas.loc[df_notas['situacao'].str.contains('APROVADO'), 'situacao']  = 'APROVADO'

    #seleciona as situacoes de interesse que são aprovados e reprovados
    df_notas = df_notas.loc[(df_notas['situacao'].str.contains('APROVADO')) |
                            (df_notas['situacao'].str.contains('REPROVADO')) 
                          ] 	
    #tem disciplina que só contam para a horas complementares, não tem nota
    df_notas = df_notas.loc[ ~(df_notas['nota'].str.contains('-')), :]

    #conversao para de string para float
    df_notas['nota'] = df_notas['nota'].astype(float)
    return df_notas


  def converteString(self, df_notas):
    df_notas['matricula'] = df_notas['matricula'].astype(str)
    df_notas['periodo']   = df_notas['periodo'].astype(str)
    return df_notas


  def concatenaSemestre(self, df_notas):
    #pega o ano mais alto que tem na matricula, indicando o semestre do df
    ano_df = df_notas['matricula'].str[0:4].max()
    df_notas['semestre'] = str(ano_df) + '.' + str(df_notas['periodo'].iloc[0])

    #apagando a coluna periodo, nao serve mais
    df_notas.drop(columns = ['periodo'], inplace=True)
    return df_notas


  #essa função vai criar as novas features com o desempenho do aluno no semestre 
  def geraFeaturesComDadosGenericos(self, df_notas):
    df_notas = df_notas.merge(self.carga, on = 'codigo')
    df = pd.DataFrame()
    final = pd.DataFrame()

    #media das disciplinas
    df = df_notas.groupby('matricula')['nota'].mean().reset_index()
    df.rename(columns={'nota':'mediaNotas'}, inplace = True)
    final = df

    #menor nota
    df = df_notas.groupby('matricula')['nota'].min().reset_index()
    df.rename(columns = {'nota':'menorNota'}, inplace = True)
    final = final.merge(df, on = 'matricula', how = 'outer')

    #maior nota
    df = df_notas.groupby('matricula')['nota'].max().reset_index()
    df.rename(columns = {'nota':'maiorNota'}, inplace = True)
    final = final.merge(df, on = 'matricula', how = 'outer')

    #quantidade de disciplinas cursadas
    df = df_notas.groupby('matricula')['codigo'].count().reset_index()
    df.rename(columns = {'codigo':'quantidadeDisciplinasCursadas'}, inplace=True)
    final = final.merge(df, on = 'matricula', how = 'outer')

    #quantidade de situacoes
    df = df_notas.groupby(['matricula','situacao'])['codigo'].count().reset_index()
    df.rename(columns = {'codigo':'quanSituacao'}, inplace = True)

    #quantidade de situacoes em que houve aprovacao
    aprovado = df.loc[df['situacao'] == 'APROVADO',:]
    aprovado.rename(columns = {'quanSituacao':'quantidadeAprovacoes'}, inplace = True)
    final = final.merge(aprovado, on = 'matricula', how = 'outer')

    #quantidade de situacoes em que houve reprovacao
    reprovado = df.loc[df['situacao'] == 'REPROVADO',:]
    reprovado.rename(columns = {'quanSituacao':'quantidadeReprovacoes'}, inplace = True)
    final = final.merge(reprovado, on = 'matricula', how = 'outer')
    final = final.drop(['situacao_x', 'situacao_y'], axis = 1)

    #media das aprovacoes
    aprov = df_notas.loc[df_notas['situacao'] == 'APROVADO',:]
    dfa = aprov.groupby(['matricula'])['nota'].mean().reset_index()
    dfa.rename(columns = {'nota':'mediaAprovadas'}, inplace = True)
    final = final.merge(dfa, on = 'matricula', how ='outer')

    #media das reprovacoes
    reprov = df_notas.loc[df_notas['situacao'] == 'REPROVADO',:]
    dfr = reprov.groupby(['matricula'])['nota'].mean().reset_index()
    dfr.rename(columns = {'nota':'mediaReprovadas'}, inplace = True)
    final = final.merge(dfr, on = 'matricula', how ='outer')

    #carga horaria total no semestre
    df = df_notas.groupby('matricula')['ch_total'].sum().reset_index()
    df.rename(columns = {'ch_total':'cargaHoraria'}, inplace = True)
    final = final.merge(df, on = 'matricula', how = 'outer')

    #substituindo valores nulos por 0
    final = final.fillna(0)

    #conversao de campos de objeto para inteiro
    final['quantidadeAprovacoes']  = final['quantidadeAprovacoes'].astype(int)
    final['quantidadeReprovacoes'] = final['quantidadeReprovacoes'].astype(int)

    #Criando campo taxa de sucesso, que será a porcentagem de aprovações dado o total de disciplinas
    final['taxaDeSucesso']  = np.divide(final['quantidadeAprovacoes'], final['quantidadeDisciplinasCursadas']) 

    #herdando o semestre
    final['semestre'] = df_notas['semestre']

    return final

  #--------------------fim das funcoes de geracao de dados da classe-----------------------------------------


  def transform(self, X, y = None):
    #itera a lista
    lista_df_notas = X.copy()
    lista_df_generico = []
    for df_notas in lista_df_notas:
      df_notas = self.converteString(df_notas)
      df_notas = self.filtraSituacoes(df_notas)
      df_notas = self.concatenaSemestre(df_notas)
      df_generico = self.geraFeaturesComDadosGenericos(df_notas)
      lista_df_generico.append(df_generico)
    return lista_df_generico

##3. Funções
<a id='funcoes'></a>

Função que usa GridSearch para avaliar o melhor modelo disponível, retornar estatísticas dos resultados e persistir em disco os modelos gerados

In [0]:
def avaliar_modelos(df, hyperparameters):

  #Dicionários
  model_param_disciplinas = {}
  best_acc_disciplinas = {}
  scoring_disciplinas = {}
  #feature_importances_disciplinas = {}

  #Seleciona o codigo de cada disciplina no dataframe, --alteravel--
  lista_disciplinas = df["disciplina"].unique()  

  #Verifica se esta vazio
  if lista_disciplinas is None:
	  return None

  #Convertendo categorias do target de string para numerico (0,1)
  df["situacao_categoria"] = pd.Categorical(df["situacao"]).codes

  #Pipeline para transformar em categoria e verificar melhores parametros
  pipeline_transform = Pipeline(steps = [('num_pipeline', NumericalTransformer()),
                                         ('fs',SelectKBest(chi2)),
                                         ('sc',StandardScaler()),
                                         ('clf',XGBClassifier())
                               ])

  #Grid para testar todos os parâmetros                               
  grid_search = GridSearchCV(
                          estimator = pipeline_transform, 
                          param_grid = hyperparameters,
                          cv= 5,
                          scoring = {'Accuracy': make_scorer(accuracy_score)},
                          #scoring = {"AUC": "roc_auc", "Accuracy": make_scorer(accuracy_score), "F1":"f1", "Recall":"recall", "Precision":"precision"},
                          return_train_score=True,
                          n_jobs=-1,#indica o numero de processos em paralelo, -1 significa usar todos
                          refit='Accuracy')


  #itera sobre a lista de disciplinas
  for disciplina in lista_disciplinas:
    data_iter = df.loc[df["disciplina"].str.contains(disciplina),:]
    
    #Treinando Grid
    best_model = grid_search.fit(data_iter.drop(columns = ["disciplina","situacao_categoria"]), data_iter["situacao_categoria"])
 
    #Extraindo informações do treino
    param_modelo = best_model.best_params_
    model_param_disciplinas[disciplina] = param_modelo['clf']
    best_acc_disciplinas[disciplina] = best_model.best_score_

    #Resultado de metricas
    result = pd.DataFrame(best_model.cv_results_)
    result = result[['mean_train_Accuracy', 'std_train_Accuracy','mean_test_Accuracy', 'std_test_Accuracy','rank_test_Accuracy']].copy()
    result["std_ratio"] = result.std_test_Accuracy/result.std_train_Accuracy
    result = result.sort_values(by="rank_test_Accuracy",ascending=True)    
    scoring_disciplinas[disciplina] = result.iloc[0,:]

    #Características mais importantes
    #feature_importances_disciplinas[disciplina] = grid.best_estimator_.feature_importances_
    
    #Salvando modelo
    joblib.dump(best_model, 'modelo_' + disciplina + '.pkl')


  #Transformando o dicionário para Dataframe
  df_best_acc = pd.DataFrame.from_dict(best_acc_disciplinas, orient="index")
  df_best_acc.columns = ["accuracy"]
  df_scor_disc = pd.DataFrame.from_dict(scoring_disciplinas, orient="index")

  #Concatenando DataFrames
  df_relatorio = pd.concat([df_best_acc, df_scor_disc], axis=1, sort=False)

  return (df_relatorio,model_param_disciplinas)

##4. Leitura das Bases
<a id='leiturabase'></a>

Urls das notas e da carga horária

In [0]:
notas_url = [
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2015.1.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2015.2.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2016.1.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2016.2.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2017.1.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2017.2.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2018.1.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2018.2.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2019.1.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2019.2.csv'
            ]
carga_horaria = 'http://dados.ufrn.br/dataset/3fea67e8-6916-4ed0-aaa6-9a8ca06a9bdc/resource/9a3521d2-4bc5-4fda-93f0-f701c8a20727/download/componentes-curriculares-presenciais.csv'

Lendo como DataFrame

In [0]:
col = ["periodo","codigo","disciplina","turma","siape","horario","matricula","nota","situacao"]
notas2015_1 = pd.read_csv(notas_url[0], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2015_2 = pd.read_csv(notas_url[1], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2016_1 = pd.read_csv(notas_url[2], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2016_2 = pd.read_csv(notas_url[3], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2017_1 = pd.read_csv(notas_url[4], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2017_2 = pd.read_csv(notas_url[5], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2018_1 = pd.read_csv(notas_url[6], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2018_2 = pd.read_csv(notas_url[7], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2019_1 = pd.read_csv(notas_url[8], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
#notas2019_2 = pd.read_csv(notas_url[9], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)

carga = pd.read_csv(carga_horaria, sep=";")
carga = carga[["codigo","ch_total"]]

#Parte III - Desenvolvimento
<a id='enten_dados'></a>


## 1. Entendimento de Negócio
<a id='negocio'></a>

#### ● Determinar objetivos de Negócio
#### ● Estabelecer Critérios de Sucesso da Iniciativa
#### ● Análise do contexto: terminologia, benefício, riscos e plano de contingência
#### ● Determinar objetivos de Mineração de Dados
#### ● Plano do Projeto


## 2. Entendimento de Dados
<a id='enten_dados'></a>

### [2.1 Descrição de Dados](#descricao)
### [2.2 Qualidade de Dados](#qualidade)
### [2.3 Exploração de Dados](#exploracao)

###2.1 Descrição de Dados
<a id='descricao'></a>


#### <center>Dados dos Daframes de Notas:
| Semestre | Número de atributos (colunas) | Número de Registros (linhas) |
| --- | --- | --- |
| 2015.1 | 9 | --- |
| 2015.2 | 9 | |
| 2016.1 | 9 | |
| 2016.2 | 9 | |
| 2017.1 | 9 | |
| 2017.2 | 9 | |
| 2018.1 | 9 | |
| 2018.2 | 9 | |
| 2019.1 | 9 | |


#### <center>Dados do Dataframe de Carga Horária:
| Número de atributos (colunas) | Número de Registros (linhas) |
| --- | --- |
| 2 | 39269 |




####Descrição de Colunas do Dataframe de Notas:

<font color='blue'> <b>1. periodo</b> </font> Período em que a disciplina foi cursada (.1 ou .2)

<font color='blue'> <b>2. codigo</b> </font> Código de todas as disciplinas na UFRN

<font color='blue'> <b>3. disciplina</b> </font> Nome da discplina 

<font color='blue'> <b>4. turma</b> </font> Código da turma

<font color='blue'> <b>5. siape</b> </font> Código que identifica o professor que ministrou a disciplina

<font color='blue'> <b>6. horario</b> </font> Horário e intervalo de datas em que a disciplina foi ministrada

<font color='blue'> <b>7. Matrícula</b> </font> Código identificador do estudante

<font color='blue'> <b>8. Nota</b> </font> Nota do estudante na disciplina

#### Coluna Alvo
<font color='red'> <b>9. Situação</b> </font> Estato final na disciplina

####Descrição de Colunas do Dataframe de Notas:

<font color='blue'> <b>1. Código</b> </font> Código da disciplina

<font color='blue'> <b>2. Ch_total</b> </font> Carga Hóraria da displina

-------------------------------------------------------------------------------

Os Dataframes contêm todos os departamentos da UFRN. Por enquanto o foco do trabalho está em prever o ciclo básico da ECT. Mais na frente será feito uma seleção afim de escolher apenas matérias da ECT.



Exemplos de registros nos 2 tipos de DataFrames

In [0]:
notas2018_1.sample(10)

In [0]:
carga.head(2)

###2.2 Qualidade dos dados
<a id='qualidade'></a>

Total 489 disciplinas realizadas na UFRN no semestre 15.1

In [0]:
notas2015_1.groupby('codigo')['nome'].count()

Informações sobre valores não nulos, total de registros de todos as bases de dados

Em quase todos os dataframes das notas não contêm valores nulos, com excessão de 16.2, 18.2 e 19.1, onde estão faltando em cada 3 valores correspondentes aos horários das displinas. Ou seja, temos dataframes consistentes

No dataframe de cargas horárias também temos consistência

In [0]:
notas2015_1.info()

In [0]:
notas2015_2.info()

In [0]:
notas2016_1.info()

In [0]:
notas2016_2.info()

In [0]:
notas2017_1.info()

In [0]:
notas2017_2.info()

In [0]:
notas2018_1.info()

In [0]:
notas2018_2.info()

In [0]:
notas2019_1.info()

In [0]:
carga.info()

###2.3 Exploração de Dados
<a id='exploracao'></a>

Temos várias situações disponíveis ao fim da matéria. Excluída, Trancada, Desistência, Indiferido e Cancelado não farão parte da nossa análise que busca apenas alunos que foram do começo ao fim da matéria.

Todos os tipos de reprovação iram virar uma só, assim como às aprovações.

In [0]:
notas2015_1['situacao'].unique()

Vemos que existe disciplinas em que a nota média final foi '-'. Como pode?

A primeira hipótese seria de que algúem zerou todas as unidades e saiu com nota 0, e o sistema colocaria '-' 

Descobri que existem disciplinas que contam apenas para carga horária complementar. Ou seja, não tem uma nota final para o estudante

Além disso vemos que as notas estão em objeto (string), vai ser necessário a conversão para float

In [0]:
notas2015_1['nota'].unique()

##3. Preparação de Dados
<a id='preparo'></a>

Adicionando os DF a uma lista para minimizar o número de linhas

In [0]:
lista_df_notas = [notas2015_1, notas2015_2, notas2016_1, notas2016_2 , notas2017_1, notas2017_2, notas2018_1, notas2018_2, notas2019_1]

In [0]:
lista_df_notas[-1].iloc[:2,1:]

In [5]:
lista_df_notas[-1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20620 entries, 0 to 20619
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   periodo     20620 non-null  int64 
 1   codigo      20620 non-null  object
 2   disciplina  20620 non-null  object
 3   turma       20620 non-null  object
 4   siape       20620 non-null  object
 5   horario     20617 non-null  object
 6   matricula   20620 non-null  int64 
 7   nota        20620 non-null  object
 8   situacao    20620 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.4+ MB


In [0]:
ano_df = lista_df_notas[0]['matricula'].str[0:4].max()
ano_df

In [0]:
lista_df_notas[0]

In [0]:
lista_copia = []
lista_copia_copia = []
lista_copia = lista_df_notas
lista_copia_copia = lista_copia

In [48]:
#antes de chamar a funcao/pipeline, 
#table = table.merge(carga,on="codigo")

lista_retorno = []
lista_retorno = geraDadosGenericos(lista_df_notas, carga)
                   

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
lista_retorno[0]

In [0]:
def geraDadosGenericos(X, carga):
  #itera a lista
  lista_df_notas = X.copy()
  lista_df_generico = []
  for df_notas in lista_df_notas:
    df_notas = converteParaString(df_notas)
    df_notas = filtraSituacoes(df_notas)
    df_notas = concatenaSemestre(df_notas)
    df_generico = geraFeaturesComDadosGenericos(df_notas, carga)
    lista_df_generico.append(df_generico)
  return lista_df_generico

In [0]:
def converteParaString(df_notas):
  df_notas['matricula'] = df_notas['matricula'].astype(str)
  df_notas['periodo']   = df_notas['periodo'].astype(str)
  return df_notas

In [0]:
def concatenaSemestre(df_notas):
  #pega o ano mais alto que tem na matricula, indicando o ano do df
  ano_df = df_notas['matricula'].str[0:4].max()
  df_notas['semestre'] = str(ano_df) + '.' + str(df_notas['periodo'].iloc[0])

  #apagando a coluna periodo, nao serve mais
  df_notas.drop(columns = ['periodo'], inplace=True)

  return df_notas

In [0]:
def filtraSituacoes(df_notas):
  #substitui todas as situacoes de reprovado e aprovado para ser apenas um tipo
  df_notas.loc[df_notas['situacao'].str.contains('REPROVADO'), 'situacao'] = 'REPROVADO'
  df_notas.loc[df_notas['situacao'].str.contains('APROVADO'), 'situacao'] = 'APROVADO'

  #seleciona as situacoes de interesse que são aprovados e reprovados
  df_notas = df_notas.loc[(df_notas['situacao'].str.contains('APROVADO')) |
                          (df_notas['situacao'].str.contains('REPROVADO')) 
                         ]
  #tem disciplina que só contam para a horas complementares, não tem nota       
  df_notas = df_notas.loc[ ~(df_notas['nota'].str.contains('-')), :]   

  #conversao para de string para float   
  df_notas['nota'] = df_notas['nota'].astype(float)            
                                                  
  return df_notas

In [0]:
def geraFeaturesComDadosGenericos(df_notas, carga):
  df_notas = df_notas.merge(carga, on = 'codigo')
  df = pd.DataFrame()
  final = pd.DataFrame()

  #media das disciplinas
  df = df_notas.groupby('matricula')['nota'].mean().reset_index()
  df.rename(columns={'nota':'mediaNotas'}, inplace = True)
  final = df

  #menor nota
  df = df_notas.groupby('matricula')['nota'].min().reset_index()
  df.rename(columns = {'nota':'menorNota'}, inplace = True)
  final = final.merge(df, on = 'matricula', how = 'outer')

  #maior nota
  df = df_notas.groupby('matricula')['nota'].max().reset_index()
  df.rename(columns = {'nota':'maiorNota'}, inplace = True)
  final = final.merge(df, on = 'matricula', how = 'outer')

  #quantidade de disciplinas cursadas
  df = df_notas.groupby('matricula')['codigo'].count().reset_index()
  df.rename(columns = {'codigo':'quantidadeDisciplinasCursadas'}, inplace=True)
  final = final.merge(df, on = 'matricula', how = 'outer')

  #quantidade de situacoes
  df = df_notas.groupby(['matricula','situacao'])['codigo'].count().reset_index()
  df.rename(columns = {'codigo':'quanSituacao'}, inplace = True)

  #quantidade de situacoes em que houve aprovacao
  aprovado = df.loc[df['situacao'] == 'APROVADO',:]
  aprovado.rename(columns = {'quanSituacao':'quantidadeAprovacoes'}, inplace = True)
  final = final.merge(aprovado, on = 'matricula', how = 'outer')

  #quantidade de situacoes em que houve reprovacao
  reprovado = df.loc[df['situacao'] == 'REPROVADO',:]
  reprovado.rename(columns = {'quanSituacao':'quantidadeReprovacoes'}, inplace = True)
  final = final.merge(reprovado, on = 'matricula', how = 'outer')
  final = final.drop(['situacao_x', 'situacao_y'], axis = 1)

  #media das aprovacoes
  aprov = df_notas.loc[df_notas['situacao'] == 'APROVADO',:]
  dfa = aprov.groupby(['matricula'])['nota'].mean().reset_index()
  dfa.rename(columns = {'nota':'mediaAprovadas'}, inplace = True)
  final = final.merge(dfa, on = 'matricula', how ='outer')

  #media das reprovacoes
  reprov = df_notas.loc[df_notas['situacao'] == 'REPROVADO',:]
  dfr = reprov.groupby(['matricula'])['nota'].mean().reset_index()
  dfr.rename(columns = {'nota':'mediaReprovadas'}, inplace = True)
  final = final.merge(dfr, on = 'matricula', how ='outer')

  #carga horaria total no semestre
  df = df_notas.groupby('matricula')['ch_total'].sum().reset_index()
  df.rename(columns = {'ch_total':'cargaHoraria'}, inplace = True)
  final = final.merge(df, on = 'matricula', how = 'outer')

  #substituindo valores nulos por 0
  final = final.fillna(0)

  #conversao de campos de objeto para inteiro
  final['quantidadeAprovacoes']  = final['quantidadeAprovacoes'].astype(int)
  final['quantidadeReprovacoes'] = final['quantidadeReprovacoes'].astype(int)

  #Criando campo taxa de sucesso, que será a porcentagem de aprovações dado o total de disciplinas
  final['taxaDeSucesso']  = np.divide(final['quantidadeAprovacoes'], final['quantidadeDisciplinasCursadas']) 

  #herdando o semestre
  final['semestre'] = df_notas['semestre']

  return final

A idéia de geração dos dados genéricos é a seguinte

Passos de transformação de dados

* primeiro
* segundo

Classe para realizar transformação nos Dados

Ela será no formato sugerido pela Sklearn para ser compatível com pipelines


Função para montar a tabela com os antecedentes dos alunos

def criaDataFrameAntecedentes(table): #usa carga horaria




Função para inserir o ano de ingresso do aluno
def AnoIngresso(table) : 



Função para adicionar os anos a cada semestre
def concatena(table,i,ano):
--chama
filtrositu
converte
criaDFAnte

eu passo o df de notas 
retorna o df de notas
e o de semestre


['matricula', 'media', 'min', 'max', 'QuantDisciCursadas',
       'quantAprovado', 'quantReprovado', 'mediasAprovadas',
       'mediasReprovadas', 'carga_total', 'anoSemestre', 'taxaDeSucesso',
       'AnoIngresso', 'AnosMatriculado']


serao 2 classes de transformacao nesse notebook. A que treina os modelos é a proxima

depois o próximo passo é concatenar o dataframe antecedente com o resultado na materia do semestre seguinte 


bagunça.com
Função para concatenar o antecedente com a situação em matéria posterior
def concatenaAntecessoresComResultadoPosterior(tableAntecedente,tableAtual,situacaoPosterior,final,flag):



In [20]:
str_transf = geraDadosGenericos(carga)
df_teste = str_transf.transform(lista_df_notas)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
df_teste[0]

,matricula,mediaNotas,menorNota,maiorNota,quantidadeDisciplinasCursadas,quantidadeAprovacoes,quantidadeReprovacoes,mediaAprovadas,mediaReprovadas,cargaHoraria,taxaDeSucesso,semestre
0,2009028341,4.971429,0.7,9.1,7,5,2,6.540000,1.050000,450,0.714286,2015.1
1,2009028457,3.950000,0.0,6.1,4,2,2,5.900000,2.000000,210,0.500000,2015.1
2,2009028473,6.485714,5.2,8.6,7,7,0,6.485714,0.000000,420,1.000000,2015.1
3,2009029488,4.033333,0.0,7.2,6,3,3,6.266667,1.800000,360,0.500000,2015.1
4,2009029534,5.880000,2.4,7.1,5,4,1,6.750000,2.400000,270,0.800000,2015.1
...,...,...,...,...,...,...,...,...,...,...,...,...
3069,2015096155,3.883333,1.2,6.6,6,3,3,6.133333,1.633333,330,0.500000,2015.1
3070,2015096217,4.733333,0.2,7.6,6,3,3,6.733333,2.733333,330,0.500000,2015.1
3071,2015096315,3.050000,0.0,6.8,6,2,4,6.100000,1.525000,330,0.333333,2015.1
3072,2015096398,6.050000,5.4,6.7,2,2,0,6.050000,0.000000,120,1.000000,2015.1


In [0]:
class geraDadosGenericos(TransformerMixin):  
  def __init__(self, carga):
    self.carga = carga

	#nao irei fazer nenhum treinamento	


  def fit(self, X, y = None):
    return self    

  #-------------------------funcoes de geracao de dados da classe---------------------------------

  def filtraSituacoes(self, df_notas):
    #conversao para string
    df_notas['matricula'] = df_notas['matricula'].astype(str)
    df_notas['periodo']   = df_notas['periodo'].astype(str)

    #substitui todas as situacoes de reprovado e aprovado para ser apenas um tipo
    df_notas.loc[df_notas['situacao'].str.contains('REPROVADO'), 'situacao'] = 'REPROVADO'
    df_notas.loc[df_notas['situacao'].str.contains('APROVADO'), 'situacao']  = 'APROVADO'

    #seleciona as situacoes de interesse que são aprovados e reprovados
    df_notas = df_notas.loc[(df_notas['situacao'].str.contains('APROVADO')) |
                            (df_notas['situacao'].str.contains('REPROVADO')) 
                          ] 	
    #tem disciplina que só contam para a horas complementares, não tem nota
    df_notas = df_notas.loc[ ~(df_notas['nota'].str.contains('-')), :]

    #conversao para de string para float
    df_notas['nota'] = df_notas['nota'].astype(float)
    return df_notas


  def converteString(self, df_notas):
    df_notas['matricula'] = df_notas['matricula'].astype(str)
    df_notas['periodo']   = df_notas['periodo'].astype(str)
    return df_notas


  def concatenaSemestre(self, df_notas):
    #pega o ano mais alto que tem na matricula, indicando o semestre do df
    ano_df = df_notas['matricula'].str[0:4].max()
    df_notas['semestre'] = str(ano_df) + '.' + str(df_notas['periodo'].iloc[0])

    #apagando a coluna periodo, nao serve mais
    df_notas.drop(columns = ['periodo'], inplace=True)
    return df_notas


  #essa função vai criar as novas features com o desempenho do aluno no semestre 
  def geraFeaturesComDadosGenericos(self, df_notas):
    df_notas = df_notas.merge(self.carga, on = 'codigo')
    df = pd.DataFrame()
    final = pd.DataFrame()

    #media das disciplinas
    df = df_notas.groupby('matricula')['nota'].mean().reset_index()
    df.rename(columns={'nota':'mediaNotas'}, inplace = True)
    final = df

    #menor nota
    df = df_notas.groupby('matricula')['nota'].min().reset_index()
    df.rename(columns = {'nota':'menorNota'}, inplace = True)
    final = final.merge(df, on = 'matricula', how = 'outer')

    #maior nota
    df = df_notas.groupby('matricula')['nota'].max().reset_index()
    df.rename(columns = {'nota':'maiorNota'}, inplace = True)
    final = final.merge(df, on = 'matricula', how = 'outer')

    #quantidade de disciplinas cursadas
    df = df_notas.groupby('matricula')['codigo'].count().reset_index()
    df.rename(columns = {'codigo':'quantidadeDisciplinasCursadas'}, inplace=True)
    final = final.merge(df, on = 'matricula', how = 'outer')

    #quantidade de situacoes
    df = df_notas.groupby(['matricula','situacao'])['codigo'].count().reset_index()
    df.rename(columns = {'codigo':'quanSituacao'}, inplace = True)

    #quantidade de situacoes em que houve aprovacao
    aprovado = df.loc[df['situacao'] == 'APROVADO',:]
    aprovado.rename(columns = {'quanSituacao':'quantidadeAprovacoes'}, inplace = True)
    final = final.merge(aprovado, on = 'matricula', how = 'outer')

    #quantidade de situacoes em que houve reprovacao
    reprovado = df.loc[df['situacao'] == 'REPROVADO',:]
    reprovado.rename(columns = {'quanSituacao':'quantidadeReprovacoes'}, inplace = True)
    final = final.merge(reprovado, on = 'matricula', how = 'outer')
    final = final.drop(['situacao_x', 'situacao_y'], axis = 1)

    #media das aprovacoes
    aprov = df_notas.loc[df_notas['situacao'] == 'APROVADO',:]
    dfa = aprov.groupby(['matricula'])['nota'].mean().reset_index()
    dfa.rename(columns = {'nota':'mediaAprovadas'}, inplace = True)
    final = final.merge(dfa, on = 'matricula', how ='outer')

    #media das reprovacoes
    reprov = df_notas.loc[df_notas['situacao'] == 'REPROVADO',:]
    dfr = reprov.groupby(['matricula'])['nota'].mean().reset_index()
    dfr.rename(columns = {'nota':'mediaReprovadas'}, inplace = True)
    final = final.merge(dfr, on = 'matricula', how ='outer')

    #carga horaria total no semestre
    df = df_notas.groupby('matricula')['ch_total'].sum().reset_index()
    df.rename(columns = {'ch_total':'cargaHoraria'}, inplace = True)
    final = final.merge(df, on = 'matricula', how = 'outer')

    #substituindo valores nulos por 0
    final = final.fillna(0)

    #conversao de campos de objeto para inteiro
    final['quantidadeAprovacoes']  = final['quantidadeAprovacoes'].astype(int)
    final['quantidadeReprovacoes'] = final['quantidadeReprovacoes'].astype(int)

    #Criando campo taxa de sucesso, que será a porcentagem de aprovações dado o total de disciplinas
    final['taxaDeSucesso']  = np.divide(final['quantidadeAprovacoes'], final['quantidadeDisciplinasCursadas']) 

    #herdando o semestre
    final['semestre'] = df_notas['semestre']

    return final

  #--------------------fim das funcoes de geracao de dados da classe-----------------------------------------


  def transform(self, X, y = None):
    #itera a lista
    lista_df_notas = X.copy()
    lista_df_generico = []
    for df_notas in lista_df_notas:
      df_notas = self.converteString(df_notas)
      df_notas = self.filtraSituacoes(df_notas)
      df_notas = self.concatenaSemestre(df_notas)
      df_generico = self.geraFeaturesComDadosGenericos(df_notas)
      lista_df_generico.append(df_generico)
    return lista_df_generico

In [0]:
class geraDadosGenericos(TansformerMixin):  
  def __init__(self, carga):
    self.carga = carga

	#nao irei fazer nenhum treinamento	
	def fit(self, X, y = None):
    return self

  #-------------------------funcoes de geracao de dados da classe---------------------------------

  def converteParaString(df_notas):
    df_notas['matricula'] = df_notas['matricula'].astype(str)
    df_notas['periodo']   = df_notas['periodo'].astype(str)
    return df_notas


  def filtraSituacoes(self, df_notas):
    #substitui todas as situacoes de reprovado e aprovado para ser apenas um tipo
    df_notas.loc[df_notas['situacao'].str.contains('REPROVADO'), 'situacao'] = 'REPROVADO'
    df_notas.loc[df_notas['situacao'].str.contains('APROVADO'), 'situacao']  = 'APROVADO'

    #seleciona as situacoes de interesse que são aprovados e reprovados
    df_notas = df_notas.loc[(df_notas['situacao'].str.contains('APROVADO')) |
                            (df_notas['situacao'].str.contains('REPROVADO')) 
                          ] 	
    #tem disciplina que só contam para a horas complementares, não tem nota
    df_notas = df_notas.loc[ ~(df_notas['nota'].str.contains('-')), :]

    #conversao para de string para float
    df_notas['nota'] = df_notas['nota'].astype(float)
    return df_notas
    

  def concatenaSemestre(self, df_notas):
    #pega o ano mais alto que tem na matricula, indicando o semestre do df
    ano_df = df_notas['matricula'].str[0:4].max()
    df_notas['semestre'] = str(ano_df) + '.' + str(df_notas['periodo'].iloc[0])

    #apagando a coluna periodo, nao serve mais
    df_notas.drop(columns = ['periodo'], inplace=True)
    return df_notas


  #essa função vai criar as novas features com o desempenho do aluno no semestre 
  def geraFeaturesComDadosGenericos(self, df_notas):
    df_notas = df_notas.merge(self.carga, on = 'codigo')
    df = pd.DataFrame()
    final = pd.DataFrame()

    #media das disciplinas
    df = df_notas.groupby('matricula')['nota'].mean().reset_index()
    df.rename(columns={'nota':'mediaNotas'}, inplace = True)
    final = df

    #menor nota
    df = df_notas.groupby('matricula')['nota'].min().reset_index()
    df.rename(columns = {'nota':'menorNota'}, inplace = True)
    final = final.merge(df, on = 'matricula', how = 'outer')

    #maior nota
    df = df_notas.groupby('matricula')['nota'].max().reset_index()
    df.rename(columns = {'nota':'maiorNota'}, inplace = True)
    final = final.merge(df, on = 'matricula', how = 'outer')

    #quantidade de disciplinas cursadas
    df = df_notas.groupby('matricula')['codigo'].count().reset_index()
    df.rename(columns = {'codigo':'quantidadeDisciplinasCursadas'}, inplace=True)
    final = final.merge(df, on = 'matricula', how = 'outer')

    #quantidade de situacoes
    df = df_notas.groupby(['matricula','situacao'])['codigo'].count().reset_index()
    df.rename(columns = {'codigo':'quanSituacao'}, inplace = True)

    #quantidade de situacoes em que houve aprovacao
    aprovado = df.loc[df['situacao'] == 'APROVADO',:]
    aprovado.rename(columns = {'quanSituacao':'quantidadeAprovacoes'}, inplace = True)
    final = final.merge(aprovado, on = 'matricula', how = 'outer')

    #quantidade de situacoes em que houve reprovacao
    reprovado = df.loc[df['situacao'] == 'REPROVADO',:]
    reprovado.rename(columns = {'quanSituacao':'quantidadeReprovacoes'}, inplace = True)
    final = final.merge(reprovado, on = 'matricula', how = 'outer')
    final = final.drop(['situacao_x', 'situacao_y'], axis = 1)

    #media das aprovacoes
    aprov = df_notas.loc[df_notas['situacao'] == 'APROVADO',:]
    dfa = aprov.groupby(['matricula'])['nota'].mean().reset_index()
    dfa.rename(columns = {'nota':'mediaAprovadas'}, inplace = True)
    final = final.merge(dfa, on = 'matricula', how ='outer')

    #media das reprovacoes
    reprov = df_notas.loc[df_notas['situacao'] == 'REPROVADO',:]
    dfr = reprov.groupby(['matricula'])['nota'].mean().reset_index()
    dfr.rename(columns = {'nota':'mediaReprovadas'}, inplace = True)
    final = final.merge(dfr, on = 'matricula', how ='outer')

    #carga horaria total no semestre
    df = df_notas.groupby('matricula')['ch_total'].sum().reset_index()
    df.rename(columns = {'ch_total':'cargaHoraria'}, inplace = True)
    final = final.merge(df, on = 'matricula', how = 'outer')

    #substituindo valores nulos por 0
    final = final.fillna(0)

    #conversao de campos de objeto para inteiro
    final['quantidadeAprovacoes']  = final['quantidadeAprovacoes'].astype(int)
    final['quantidadeReprovacoes'] = final['quantidadeReprovacoes'].astype(int)

    #Criando campo taxa de sucesso, que será a porcentagem de aprovações dado o total de disciplinas
    final['taxaDeSucesso']  = np.divide(final['quantidadeAprovacoes'], final['quantidadeDisciplinasCursadas']) 

    #herdando o semestre
    final['semestre'] = df_notas['semestre']

    return final

  #--------------------fim das funcoes de geracao de dados da classe-----------------------------------------


  def transform(self, X, y = None):
    #itera a lista
    lista_df_notas = X.copy()
    lista_df_generico = []
    for df_notas in lista_df_notas:
      df_notas = converteParaString(df_notas)
      df_notas = filtraSituacoes(df_notas)
      df_notas = concatenaSemestre(df_notas)
      df_generico = geraFeaturesComDadosGenericos(df_notas, carga)
      lista_df_generico.append(df_generico)
    return lista_df_generico

In [0]:
class NumericalTransformer(BaseEstimator, TransformerMixin):
  def __init__(self, model=0):
     self.model = model

  def fit(self, X, y = None ):
    return self 

  def create_dummies(self, df, column_name, drop_first_col):
    #Cria colunas fictícias apartir de uma
    dummies = pd.get_dummies(df[column_name],prefix=column_name, drop_first=drop_first_col)
    return dummies

  #Manipulando a coluna "media"
  def process_media(self, df):
    #divide a coluna de média em um intervalo de valores 
    cut_points_notas = [-1,3,5,7,8,11]
    label_names_notas = ["muito baixo","baixo","media","alto","muito alto"]
    df["media_categorias"] = pd.cut(df["media"],
                                    cut_points_notas,
                                    labels=label_names_notas)
    return self.create_dummies(df,"media_categorias",False)
   
  #Manipulando a coluna "min"
  def process_min(self, df):
    #divide a coluna min em um intervalo de valores 
    cut_points_notas = [-1,3,5,7,8,11]
    label_names_notas = ["muito baixo","baixo","media","alto","muito alto"]
    df["min_categorias"] = pd.cut(df["min"],
                                  cut_points_notas,
                                  labels=label_names_notas)
    return self.create_dummies(df,"min_categorias",False) 

  #Manipulando a coluna "max"
  def process_max(self, df):
    #divide a coluna max em um intervalo de valores 
    cut_points_notas = [-1,3,5,7,8,11]
    label_names_notas = ["muito baixo","baixo","media","alto","muito alto"]
    df["max_categorias"] = pd.cut(df["max"],
                                  cut_points_notas,
                                  labels=label_names_notas)
    return self.create_dummies(df,"max_categorias",False) 

  #Manipulando a coluna "carga_total"
  def process_carga_total(self, df):
    #divide a coluna carga_total em um intervalo de valores 
    cut_points_carga = [0,100,200,300,400,600]
    label_names_carga = ["muito baixo","baixo","media","alto","muito alto"] 
    df["carga_total_categorias"] = pd.cut(df["carga_total"],
                                          cut_points_carga,
                                          labels=label_names_carga)
    return self.create_dummies(df,"carga_total_categorias",False) 

  #Manipulando a coluna "carga_total_atual"
  def process_carga_total_atual(self, df):
    #divide a coluna carga_total_atual em um intervalo de valores 
    cut_points_carga = [0,100,200,300,400,600]
    label_names_carga = ["muito baixo","baixo","media","alto","muito alto"] 
    df["carga_total_atual_categorias"] = pd.cut(df["carga_total_atual"],
                                                cut_points_carga,
                                                labels=label_names_carga)
    return self.create_dummies(df,"carga_total_atual_categorias",False) 
  
  #Criando a coluna de Taxa de sucesso
  def tax_suc(self, df):
    df["tax_suc"] = df["quantAprovado"] / df["QuantDisciCursadas"]
    return pd.DataFrame(df["tax_suc"],columns=["tax_suc"])

  #Concatenando colunas
  def addcol(sefl, df):
    QuantDisciCursadas = 	df["QuantDisciCursadas"]
    quantAprovado      = 	df["quantAprovado"]
    quantReprovado     =  df["quantReprovado"]
    QuantDisciAtual    =  df["QuantDisciAtual"]
    AnosMatriculado    =  df["AnosMatriculado"]
    return pd.concat([QuantDisciCursadas, quantAprovado, quantReprovado, QuantDisciAtual,AnosMatriculado ],axis=1)


  #Método transformador
  def transform(self, X , y = None ):
    df = X.copy()
    media = self.process_media(df)  
    min1 = self.process_min(df)
    max1 = self.process_max(df)
    carga_total = self.process_carga_total(df)
    carga_total_atual = self.process_carga_total_atual(df)
    tax_suc = self.tax_suc(df)
    ad = self.addcol(df)
      
    return pd.concat([media,min1,max1,carga_total,carg

##6. Avaliação dos Modelos
<a id='avaliacao'></a>

### [6.1 Chamada da Função](#chamada)
### [6.2 Teste de Modelo](#teste)

###6.1 Chamada da Função
<a id='chamada'></a>

In [0]:
(df_relatorio, model_param_disciplinas) = avaliar_modelos(df, hyperparameters)

In [0]:
df_relatorio.head()

In [0]:
model_param_disciplinas

###6.2 Teste de Modelo
<a id='teste'></a>

Importando Modelo

In [0]:
modelo_teste = joblib.load('modelo_.pkl')

Aplicando a transformação no teste

In [0]:
num_transf = NumericalTransformer()
df_teste = model.transform(df)

In [0]:
df_teste.head(2)

Realizando teste de um modelo, prevendo probabilidade de ser aprovado

In [0]:
df_teste.iloc[42,:]

In [0]:
df_teste.info()

In [0]:
modelo_teste.predict(df_teste.iloc[42,:])

##7. Conclusão e Recomendações
<a id='conclusao'></a>

##8. Implantação de Modelos
<a id='implantacao'></a>